In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from scipy.stats import chi2
from scipy.stats import chi2_contingency
np.random.seed(10)

# Load the datasets

In [56]:
# Load datasets with correct date time format
price_demand = pd.read_csv('price_demand_data.csv', parse_dates=['SETTLEMENTDATE'], dayfirst=True)
weather = pd.read_csv('weather_data.csv', parse_dates=['Date'], dayfirst=True)

#price_demand
#price_demand.info()
#weather
#weather.info()

# Cleaning Weather dataset

In [57]:
# DATA imputation: convert 'Calm' to 0 for wind speed columns
weather = weather.replace(to_replace='Calm', value=0)

# Fill NaN with mean value for numeric cells
weather = weather.fillna(round(weather.mean(),1))

C:\Users\lhong\AppData\Local\Temp\ipykernel_21448\994420789.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  weather = weather.fillna(round(weather.mean(),1))
C:\Users\lhong\AppData\Local\Temp\ipykernel_21448\994420789.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  weather = weather.fillna(round(weather.mean(),1))


# Wraggling Price and Demand dataset

In [82]:
# Replace PRICECATEGORY values with Numeric values in new column PRICE_NUMERIC
price_demand['PRICECATEGORY'].replace( {'LOW' : 0, 'MEDIUM' : 1, 'HIGH' : 2, 'EXTREME': 3 }, inplace=True)

# Extract Date and Time seperately from SETTLEMENTDATE in date time format
price_demand['Date'] = pd.to_datetime(price_demand['SETTLEMENTDATE'].dt.date)
price_demand['Time'] = price_demand['SETTLEMENTDATE'].dt.time

# Set 'Day' every 48 time rows according to the definiteion from the data source company - AEMO
price_demand['Day'] = price_demand.index // 48

# Finding the maximum daily energy usage

In [84]:

demand = price_demand

max_demand = demand.groupby('Day')['TOTALDEMAND'].transform(max) == demand['TOTALDEMAND']
max_demand = demand[max_demand].reset_index()
max_demand['Max_TOTALDEMAND'] = max_demand['TOTALDEMAND']
max_demand 

,index,REGION,SETTLEMENTDATE,TOTALDEMAND,PRICECATEGORY,Date,Time,Day,Max_TOTALDEMAND
0,34,VIC1,2021-01-01 17:30:00,5019.64,0,2021-01-01,17:30:00,0,5019.64
1,81,VIC1,2021-01-02 17:00:00,4964.35,0,2021-01-02,17:00:00,1,4964.35
2,132,VIC1,2021-01-03 18:30:00,4503.31,0,2021-01-03,18:30:00,2,4503.31
3,180,VIC1,2021-01-04 18:30:00,4764.18,0,2021-01-04,18:30:00,3,4764.18
4,225,VIC1,2021-01-05 17:00:00,4800.64,0,2021-01-05,17:00:00,4,4800.64
...,...,...,...,...,...,...,...,...,...
238,11461,VIC1,2021-08-27 19:00:00,6769.89,2,2021-08-27,19:00:00,238,6769.89
239,11509,VIC1,2021-08-28 19:00:00,5716.32,1,2021-08-28,19:00:00,239,5716.32
240,11557,VIC1,2021-08-29 19:00:00,6227.89,3,2021-08-29,19:00:00,240,6227.89
241,11604,VIC1,2021-08-30 18:30:00,6072.91,1,2021-08-30,18:30:00,241,6072.91


# Finding the maximum daily price category

In [60]:
price = price_demand

max_price = price.groupby('Day')['PRICECATEGORY'].max()
max_price = max_price.reset_index()
max_price['Max_PRICECATEGORY'] = max_price['PRICECATEGORY']
max_price = max_price.drop(columns = ['PRICECATEGORY'])
max_price

,Day,Max_PRICECATEGORY
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
238,238,3
239,239,1
240,240,3
241,241,2


In [79]:
weather.set_index('Date')
max_demand.set_index('Date')
all_data = max_demand.merge(max_price)
all_data = all_data.merge(weather)

all_data = all_data.drop(columns = ['index','REGION','SETTLEMENTDATE'])

In [80]:
all_data

,TOTALDEMAND,PRICECATEGORY,Date,Time,Day,Max_TOTALDEMAND,Max_PRICECATEGORY,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,5019.64,0,2021-01-01,17:30:00,0,5019.64,0,15.6,29.9,0.0,...,6,N,2,1018.8,28.1,43,5.0,E,13,1015.3
1,4964.35,0,2021-01-02,17:00:00,1,4964.35,0,18.4,29.0,0.0,...,7,NNW,17,1013.3,28.7,38,7.0,SW,4,1008.5
2,4503.31,0,2021-01-03,18:30:00,2,4503.31,0,17.0,26.2,12.6,...,8,WSW,4,1007.7,23.5,59,4.0,SSW,2,1005.2
3,4764.18,0,2021-01-04,18:30:00,3,4764.18,0,16.0,18.6,2.6,...,8,SSE,11,1010.0,18.2,82,8.0,SSW,17,1011.0
4,4800.64,0,2021-01-05,17:00:00,4,4800.64,0,15.9,19.1,11.2,...,8,SSE,13,1012.5,18.2,82,8.0,SSE,19,1013.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,6769.89,2,2021-08-27,19:00:00,238,6769.89,3,4.6,13.6,0.0,...,7,,0,1020.0,12.8,65,7.0,SSE,7,1017.4
239,5716.32,1,2021-08-28,19:00:00,239,5716.32,1,5.3,17.8,0.0,...,1,N,7,1018.6,17.4,31,3.0,NNW,24,1013.5
240,6227.89,3,2021-08-29,19:00:00,240,6227.89,3,9.1,16.2,0.6,...,7,N,13,1011.4,12.8,84,7.0,S,6,1010.4
241,6072.91,1,2021-08-30,18:30:00,241,6072.91,2,6.4,17.6,4.0,...,7,N,15,1016.1,16.8,45,1.0,NNW,28,1013.2
